# Peaknet API demo for PSANA

This notebook demonstrates how to utilze Peaknet API to train and test on PSANA data

Author: Po-Nan Li (Stanford EE and SLAC)

Last update: 2019/2/28


## Install

``` shell
conda create python=2.7 h5py jupyter -p ~/space/envs/peaknet
conda install psana-conda=1.4.2 -c lcls-rhel
conda install pytorch=0.1.12 torchvision cuda80 -c soumith
conda install tensorboardX
conda install pandas
```


## Import packages

In [1]:
#%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import psana
import torch
# import torchvision
import numpy as np
import time
from peaknet import Peaknet
from peaknet_utils import *
import matplotlib.pyplot as plt
import matplotlib.patches as pat
import matplotlib.cm as cm

/reg/neh/home/liponan/space/envs/peaknet/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data

### Load data from psana

In [2]:
exp_name = "cxic0415"
# run = 90
# event_idx = 3963
run = 91
event_idx = 4633
# run = 95
# event_idx = 12159 # 12159, 13012

ds = psana.DataSource("exp=" + exp_name + ":run=" + str(run) + ":idx")
det = psana.Detector('DscCsPad')
this_run = ds.runs().next()
times = this_run.times()
num_events = len(times)
print("run", run, "number of events available", num_events)
env = ds.env()

evt = this_run.event(times[event_idx])
calib = det.calib(evt) * det.mask(evt, calib=True, status=True,
                          edges=True, central=True,
                          unbond=True, unbondnbrs=True)
print("calib", calib.shape)
print("max", calib.max())
print("min", calib.min())

('run', 91, 'number of events available', 5164)
('calib', (32, 185, 388))
('max', 14687.662)
('min', -1008.55444)


### Build labels (train)

In [3]:
# filename = "/reg/neh/home/liponan/data/cxic0415/r0090/cxic0415_0090.cxi" # 23817
# filename = "/reg/neh/home/liponan/data/cxic0415/r0095/cxic0415_0095.cxi" # 4239
filename = "/reg/neh/home/liponan/data/cxic0415/r0091/cxic0415_0091_streak.cxi" # 1635
t0 = time.time()
labels, eventIdxs = load_cxi_labels( filename, total_size=-1 )
t1 = time.time()
print("took " + str(t1-t0) + " s to build the training dataset")
print("labels", len(labels), len(labels[0]))

hits: 1365
took 0.390300989151 s to build the training dataset
('labels', 1365, 6)


### Build labels (validate)

In [4]:
# filename = "/reg/neh/home/liponan/data/cxic0415/r0090/cxic0415_0090.cxi"
# filename = "/reg/neh/home/liponan/data/cxic0415/r0095/cxic0415_0095.cxi"
filename = "/reg/neh/home/liponan/data/cxic0415/r0091/cxic0415_0091_streak.cxi"
t0 = time.time()
val_labels, val_eventIdxs = load_cxi_labels( filename, total_size=10 )
t1 = time.time()
print("took " + str(t1-t0) + " s to build the val dataset")
print("val_labels", len(val_labels), len(val_labels[0]))

hits: 1365
took 0.00816583633423 s to build the val dataset
('val_labels', 10, 6)


### Test data loader

In [5]:
imgs = psana_img_loader(eventIdxs, 0, 5, det, this_run, times)
print("imgs shape", imgs.shape)

('imgs shape', (5, 32, 185, 388))


## Network

In [7]:
project_name = "runs/cxic0415_91/2cls/ada/lr_0.0001"
n_ep = 1
init_lr = 0.0001
lr_policy = 4
macro_batch_size = 3
# algo = "sgd" #"adagrad"
algo = "adagrad"

params = {"n_ep":n_ep, "lr": init_lr, "batch_size":macro_batch_size, "lr_policy": lr_policy, 
          "train_size": len(labels), "optim": algo}

net = Peaknet()
net.loadCfg( "/reg/neh/home/liponan/ai/pytorch-yolo2/cfg/newpeaksv10-asic-2cls.cfg" )
# net.loadWeights( "../pytorch-yolo2/cfg/newpeaksv10-asic.cfg", 
#                 "../darknet/backup/newpeaksv10_500.weights")
net.init_model()
net.model.cuda()

Darknet (
  (models): ModuleList (
    (0): Sequential (
      (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
      (leaky1): LeakyReLU (0.1, inplace)
    )
    (1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (2): Sequential (
      (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (leaky2): LeakyReLU (0.1, inplace)
    )
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Sequential (
      (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (leaky3): LeakyReLU (0.1, inplace)
    )
    (5): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (6): Sequential (
      (conv4): Conv2d(128, 256, kernel_size=(1, 1), stride=(

## Train

In [8]:
n_ep = 1
n_iters = int( np.ceil( len(labels) / float(macro_batch_size) ) )
print("# iters per epoch", n_iters)
net.set_writer(project_name=project_name, parameters=params)
my_lr = init_lr

n_check = 100

debug = False



for ep in range(1, n_ep+1):
    print("============= EPOCH {} ==========".format(ep))
    if ep % lr_policy == 0:
        my_lr /= 2
        print("learning rate is now", my_lr)
    
    for i in range(n_iters):
        idx_offset = i * macro_batch_size
        if i == (n_iters-1):
            n = len(labels) - i*macro_batch_size
            batch_imgs = psana_img_loader(eventIdxs, idx_offset, n, det, this_run, times)
            batch_labels = labels[(i*macro_batch_size):]
#             batch_labels = []
        else:
            batch_imgs = psana_img_loader(eventIdxs, idx_offset, macro_batch_size, det, this_run, times)
            batch_labels = labels[i*macro_batch_size:(i+1)*macro_batch_size]
#             print( len(batch_labels), len(batch_labels[0]), batch_labels[0][0].size() )
#             batch_labels = [ (np.zeros((0,)), np.zeros((0,)), np.zeros((0,)), np.zeros((0,)), np.zeros((0,)), np.zeros((0,))) ]
#         net.set_optimizer(adagrad=(algo=="adagrad"), lr=my_lr )
        batch_imgs[ batch_imgs < 0 ] = 0
        batch_imgs = batch_imgs / batch_imgs.max()
        net.train( batch_imgs, batch_labels, mini_batch_size=64 )
        
#         net.optimize()
        if debug and i == 0:
            x_cpu = batch_imgs
            y_cpu = batch_labels
            break
        
        if (i+1) % n_check == 0:
            batch_imgs = nor_fac * psana_img_loader(eventIdxs, 600, 10, det, this_run, times)
            batch_labels = labels[(300*macro_batch_size):]
            print("==================== VAL ====================")
            recall = net.validate( batch_imgs, batch_labels, mini_batch_size=32 )
            print("VAL recall: {}".format(round(recall*100)))
            print("=============================================")

('# iters per epoch', 455)
============= EPOCH 1 ==========
32: nGT 46, recall 2, proposals 252, loss: x 3.857753, y 2.241547, w 4.619479, h 8.439746, conf 5480.581543, cls 20.870281, total 5520.610352
64: nGT 2, recall 0, proposals 0, loss: x 0.036912, y 0.064031, w 0.013032, h 0.015172, conf 5335.533691, cls 1.168513, total 5336.831543
96: nGT 38, recall 5, proposals 0, loss: x 2.252197, y 1.720590, w 2.337461, h 3.975383, conf 5030.754883, cls 20.515177, total 5061.555664
128: nGT 61, recall 0, proposals 0, loss: x 4.431842, y 3.156400, w 5.326367, h 4.721647, conf 4739.954590, cls 30.087606, total 4787.678223
160: nGT 17, recall 0, proposals 0, loss: x 0.815620, y 0.899259, w 2.320006, h 3.530380, conf 4774.967285, cls 7.256791, total 4789.789062
192: nGT 45, recall 2, proposals 0, loss: x 2.746162, y 1.687702, w 3.570370, h 5.648969, conf 4398.075684, cls 17.174248, total 4428.903320
224: nGT 42, recall 2, proposals 0, loss: x 3.251583, y 1.546593, w 3.483352, h 3.593180, conf 399

1920: nGT 39, recall 7, proposals 0, loss: x 1.553401, y 1.388116, w 2.185823, h 2.969480, conf 763.984375, cls 9.597995, total 781.679260
1952: nGT 35, recall 10, proposals 170, loss: x 1.639328, y 1.328678, w 4.890078, h 5.783172, conf 738.799988, cls 8.287042, total 760.728271
1984: nGT 29, recall 5, proposals 0, loss: x 1.435497, y 1.566579, w 2.089177, h 2.923740, conf 800.725159, cls 5.061961, total 813.802124
2016: nGT 34, recall 8, proposals 200, loss: x 1.551136, y 1.821948, w 0.354411, h 0.210846, conf 718.325134, cls 3.429342, total 725.692871
2048: nGT 77, recall 18, proposals 0, loss: x 3.504767, y 3.214111, w 0.782187, h 3.207321, conf 686.957642, cls 9.964567, total 707.630554
2080: nGT 29, recall 5, proposals 0, loss: x 1.355874, y 1.170250, w 3.565257, h 5.804124, conf 715.075378, cls 8.537374, total 735.508240
2112: nGT 34, recall 7, proposals 50, loss: x 1.623473, y 1.231241, w 0.389604, h 0.133001, conf 672.380798, cls 5.578222, total 681.336365
2144: nGT 72, recall

3840: nGT 43, recall 11, proposals 39, loss: x 1.415970, y 1.416025, w 2.426932, h 2.995872, conf 181.300674, cls 5.280903, total 194.836365
3872: nGT 45, recall 4, proposals 181, loss: x 1.777800, y 2.090364, w 4.287316, h 5.790987, conf 170.802475, cls 9.305526, total 194.054474
3904: nGT 48, recall 6, proposals 69, loss: x 2.517243, y 2.160517, w 2.230453, h 2.843032, conf 182.261276, cls 4.938099, total 196.950623
3936: nGT 28, recall 7, proposals 157, loss: x 0.826931, y 1.166504, w 1.673403, h 2.879359, conf 161.307968, cls 3.661773, total 171.515945
3968: nGT 73, recall 16, proposals 97, loss: x 2.577170, y 2.660355, w 2.082121, h 3.120872, conf 165.095261, cls 5.640967, total 181.176743
4000: nGT 48, recall 14, proposals 39, loss: x 1.525851, y 1.573524, w 0.178851, h 0.093909, conf 165.994858, cls 2.958080, total 172.325073
4032: nGT 75, recall 14, proposals 11, loss: x 2.797562, y 2.988536, w 2.321107, h 3.120786, conf 160.367584, cls 5.318141, total 176.913727
4064: nGT 205,

5760: nGT 69, recall 26, proposals 181, loss: x 1.437999, y 1.558506, w 1.945822, h 3.026169, conf 43.113808, cls 3.932282, total 55.014587
5792: nGT 35, recall 15, proposals 194, loss: x 0.576737, y 0.637515, w 2.240313, h 2.526827, conf 39.586376, cls 2.966119, total 48.533886
5824: nGT 34, recall 13, proposals 35, loss: x 0.631804, y 1.089966, w 1.837173, h 2.868004, conf 34.235645, cls 5.326208, total 45.988800
5856: nGT 32, recall 10, proposals 115, loss: x 0.684738, y 0.911489, w 2.175390, h 2.648088, conf 33.253845, cls 3.406651, total 43.080196
5888: nGT 93, recall 43, proposals 175, loss: x 1.724425, y 1.971375, w 3.428775, h 4.677849, conf 40.067795, cls 5.110383, total 56.980602
5920: nGT 101, recall 16, proposals 44, loss: x 1.815696, y 2.057302, w 2.477133, h 4.183760, conf 57.659554, cls 2.419251, total 70.612694
5952: nGT 80, recall 19, proposals 101, loss: x 1.067650, y 2.211932, w 1.816943, h 3.652598, conf 37.264469, cls 3.064564, total 49.078159
5984: nGT 121, recall

7680: nGT 23, recall 10, proposals 53, loss: x 0.362120, y 0.426594, w 1.422411, h 1.970492, conf 9.094753, cls 2.058898, total 15.335267
7712: nGT 49, recall 35, proposals 105, loss: x 0.498427, y 0.708386, w 0.169431, h 0.164768, conf 15.076937, cls 0.269713, total 16.887665
7744: nGT 98, recall 50, proposals 3, loss: x 1.219351, y 1.836185, w 1.684668, h 1.730373, conf 30.520439, cls 1.609724, total 38.600739
7776: nGT 99, recall 66, proposals 148, loss: x 1.157571, y 1.155651, w 3.371241, h 4.161951, conf 26.523970, cls 3.870842, total 40.241226
7808: nGT 73, recall 40, proposals 197, loss: x 0.877757, y 1.326386, w 1.377524, h 2.028949, conf 19.816805, cls 2.538931, total 27.966351
7840: nGT 30, recall 20, proposals 89, loss: x 0.217986, y 0.275279, w 0.125922, h 0.147542, conf 11.414168, cls 0.084523, total 12.265420
7872: nGT 27, recall 15, proposals 39, loss: x 0.251144, y 0.560044, w 3.084918, h 3.427526, conf 9.895715, cls 4.137884, total 21.357233
7904: nGT 15, recall 10, pr

9600: nGT 38, recall 26, proposals 105, loss: x 0.369508, y 0.402495, w 2.448098, h 3.636169, conf 11.283429, cls 6.894949, total 25.034649


NameError: name 'nor_fac' is not defined

## Debugger for training

In [ ]:
print("x shape", x_cpu.shape)
print("y length", len(y_cpu))
print("y[0] length", len(y_cpu[0]))
print("y[0][0] shape", y_cpu[0][0].shape)

n, m, h, w = x_cpu.shape

In [ ]:
img_no = 1
panel_no = 9

fig, ax = plt.subplots(1)
im0 = plt.imshow(x_cpu[img_no, panel_no, :,:], cmap=cm.gray, vmin=0)
fig.set_size_inches(12, 6)
for j, idx in enumerate(y_cpu[img_no][1]):
    if idx == panel_no:
        pass
    else:
        continue
    ww = y_cpu[img_no][5][j] 
    hh = y_cpu[img_no][4][j]
    x = y_cpu[img_no][3][j] - 0.5*ww
    y = y_cpu[img_no][2][j] - 0.5*hh
    print(y_cpu[img_no][0][j],y_cpu[img_no][1][j],x,y,ww,hh)
    if y_cpu[img_no][0][j] == 0:
        color = 'm'
    else:
        color = 'c'
    rect = pat.Rectangle( (x, y), ww, hh, color=color, fill=False, linewidth=1 )
    ax.add_patch(rect)

## Predict

In [ ]:
imgs = calib.reshape((1,32,185,388)) / calib.max()
predict_labels = []
print("imgs", imgs.shape)
# print("labels", len(labels), len(labels[0]))

In [ ]:
results = net.predict( imgs, batch_size=32, conf_thresh=0.01, nms_thresh=0.05, use_cuda=True )


In [ ]:
print(len(results[0]), "peaks found")
print((results[0]))

In [ ]:
visualize(imgs, predict_labels, results, plot_label=False, plot_box=True, vmin=0, vmax=0.1,
          indexes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [ ]:
#a, b, c, d = build_dataset( filename, dev_size = 1, total_size=10 )